In [ ]:
!pip install pynrrd
!pip install ipywidgets
!pip install scipy

In [1]:
import os
import glob
import numpy as np
import nrrd
import timeit
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from scipy import ndimage


En el siguiente bloque definimos las funciones necesarias para realizar la extracción de imágenes del modelo 3D

In [2]:

def extract_slices(data, angle, plane):
    slices = []
    x_size, y_size, z_size = data.shape

    if plane == 'x':
        # Rotamos la imagen 3D respecto el plano x
        rotated_data = ndimage.rotate(data, angle, axes=(1, 0), reshape=True)
        for i in range(x_size):
            slices.append(rotated_data[i, :, :])
    elif plane == 'z':
        # Rotamos la imagen 3D respecto el plano z
        rotated_data_z = ndimage.rotate(data, angle, axes=(1, 2), reshape=True)
        for i in range(z_size):
            slices.append(rotated_data_z[:, :, i])

    return slices


def read_nrrd_file(filename):
    data, header = nrrd.read(filename)
    return data, header


def save_slices(slices, output_dir, filename, plane, angle):
    file_name = os.path.splitext(filename)[0]
    for i, slice_image in enumerate(slices):
        # Determinamos si son ruido, en este caso las primeras 20 y las ultimas 20.
        if i < 20 or i >= len(slices) - 20:
            folder_name = "ruido"
        else:
            folder_name = f"plane_{plane}"

        # Creamos los directorios necesarios
        plane_dir = os.path.join(output_dir, file_name, folder_name)
        angle_dir = os.path.join(plane_dir, f"angle_{angle}")
        os.makedirs(angle_dir, exist_ok=True)

        # Los nombramos como, nombre del archivo, angulo, indice.
        slice_filename = os.path.join(angle_dir, f"{file_name}_slice_{angle}_{i}.png")
        plt.imsave(slice_filename, slice_image, cmap='gray')


In [3]:

def main(input_dir, output_dir):
    # Obtenemos los archivos NRRD del directorio
    nrrd_files = glob.glob(os.path.join(input_dir, "*.nrrd"))

    # Procesamos cada archivo NRRD
    for nrrd_file in nrrd_files:
        print(f"Processing {nrrd_file}")

        # Usamos la funcion read_nrrd_file para obtener la forma y dimensiones
        data, header = read_nrrd_file(nrrd_file)
        print(f"Image dimensions: {data.shape}")
        print(f"Range of motion (x, y, z): {header['sizes']}")

        # Extraemos y guardamos los cortes
        angles = list(range(-21, 21, 5))  # realizamos los cortes cada 5 grados
        planes = ['x', 'z']  # Escogemos los panos sobre los que hacemos la rotacion

        for plane in planes:
            for angle in angles:
                slices = extract_slices(data, angle, plane)
                save_slices(slices, output_dir, os.path.basename(nrrd_file), plane, angle)


In [4]:
if __name__ == "__main__":
    input_dir = "/home/isaac/git/clasificador-imagenes-ultrasonidos/archivos_nrrd/"  # Directorio con los archivos NRRD
    output_dir = "/home/isaac/git/clasificador-imagenes-ultrasonidos/datos_Nuevoinicio"  # Directorio donde se guardarán las rodajas
    main(input_dir, output_dir)

Processing /home/isaac/git/clasificador-imagenes-ultrasonidos/archivos_nrrd/4_2013_06_30.nrrd
Image dimensions: (251, 195, 217)
Range of motion (x, y, z): [251 195 217]
Processing /home/isaac/git/clasificador-imagenes-ultrasonidos/archivos_nrrd/1_2013_11_08.nrrd
Image dimensions: (251, 187, 217)
Range of motion (x, y, z): [251 187 217]
Processing /home/isaac/git/clasificador-imagenes-ultrasonidos/archivos_nrrd/4075542_2015_10_04.nrrd
Image dimensions: (199, 203, 211)
Range of motion (x, y, z): [199 203 211]
Processing /home/isaac/git/clasificador-imagenes-ultrasonidos/archivos_nrrd/1_2013_11_15.nrrd
Image dimensions: (251, 189, 217)
Range of motion (x, y, z): [251 189 217]


Este es un paso intermedio para mover los cortes a sus planos medicos correspondientes. 

In [5]:

import shutil

# Define the directory with the current images and the new dataset directory
current_dir = "/home/isaac/git/clasificador-imagenes-ultrasonidos/datos_Nuevoinicio"
dataset_dir = "/home/isaac/git/clasificador-imagenes-ultrasonidos/datos_Nuevoinicio_org"

# Create the new directories
os.makedirs(os.path.join(dataset_dir, "plano_sagital"), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, "plano_coronal"), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, "ruido"), exist_ok=True)

# Move the images to the new directories
for root, dirs, files in os.walk(current_dir):
    for file in files:
        if file.endswith(".png"):
            if "plane_x" in root:
                shutil.move(os.path.join(root, file), os.path.join(dataset_dir, "plano_sagital", file))
            elif "plane_z" in root:
                shutil.move(os.path.join(root, file), os.path.join(dataset_dir, "plano_coronal", file))
            elif "ruido" in root:
                shutil.move(os.path.join(root, file), os.path.join(dataset_dir, "ruido", file))


Reescalamos las imagenes para que tengan todas el mismo tamaño, además las pasamos a formato RGB.

In [6]:

from PIL import Image
import os

def resize_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)
    
    for root, dirs, files in os.walk(input_dir):
        # Mantenemos la estructura de subdirectorios
        relative_path = os.path.relpath(root, input_dir)
        new_root = os.path.join(output_dir, relative_path)
        os.makedirs(new_root, exist_ok=True)
        
        for file in files:
            if file.endswith(".png"):
                input_path = os.path.join(root, file)
                output_path = os.path.join(new_root, file)
                
                image = Image.open(input_path)

                if image.mode not in ('RGB', 'L'):
                    image = image.convert('RGB')

                resized_image = image.resize(size, Image.LANCZOS)
                resized_image.save(output_path)

dataset_directory = "/home/isaac/git/clasificador-imagenes-ultrasonidos/datos_Nuevoinicio_org"
output_directory = "/home/isaac/git/clasificador-imagenes-ultrasonidos/dataset_resized_NuevoInicio"
resize_images(dataset_directory, output_directory)

